In [10]:
import sys
sys.path.append('../lsnc/')
from hierarchical_lsnc import HierarchicalLSNC
import numpy as np
from tqdm import tqdm
import os


## import the list of datasets
datasets = []
for directory in os.listdir("./labeled-datasets/npy/"):
	if (
		directory != ".DS_Store"
		and directory != "README.md"
		and directory != ".gitignore"
		and not directory.endswith(".zip")
	):
		datasets.append(directory)

## load the data
raw_list     = []
label_list   = []
tsne_list    = []
umap_list    = []
pca_list	   = []
iso_list	   = []
lle_list     = []
densmap_list = []

for dataset in tqdm(datasets):
	raw_list.append(np.load("./labeled-datasets/npy/{}/data.npy".format(dataset)))
	label_list.append(np.load("./labeled-datasets/npy/{}/label.npy".format(dataset)))
	tsne_list.append(np.load("./labeled-datasets_embedding/{}/tsne.npy".format(dataset)))
	umap_list.append(np.load("./labeled-datasets_embedding/{}/umap.npy".format(dataset)))
	pca_list.append(np.load("./labeled-datasets_embedding/{}/pca.npy".format(dataset)))
	iso_list.append(np.load("./labeled-datasets_embedding/{}/isomap.npy".format(dataset)))
	lle_list.append(np.load("./labeled-datasets_embedding/{}/lle.npy".format(dataset)))
	densmap_list.append(np.load("./labeled-datasets_embedding/{}/densmap.npy".format(dataset)))




100%|██████████| 94/94 [00:00<00:00, 162.56it/s]


In [ ]:
'''
Compare t-SNE / UMAP / PCA / Isomap / LLE / Densmap from using hierarchical LSNC (already imported)
aggregate the results as a single line for each technique 
(use lineplot in seaborn)

the final result should be a lineplot with 6 lines and confidence intervals (for each technique)
'''




